- Đơn giản hóa bài toán: tìm cách tối ưu cho 31 ngày; hết 1 vòng thì lặp lại theo đúng thứ tự cũ

- Cần tìm **thuật toán** để: thay đổi cách sắp xếp 1 cách hợp lý theo từng bước -> tăng giá trị fitness function

**Cách chọn tốt nhất hiện tại**:

In [1]:
# {'BS1': array([2, 1, 2, 3, 1, 4, 3, 3, 2, 3, 2, 2, 4, 4, 1, 4, 2, 3, 3]),
#   'BS2': array([1, 4, 4, 4, 3, 2, 4, 2, 4, 2, 1, 4, 3, 4, 3, 2, 3]),
#   'DD1': array([3, 3, 3, 1, 2, 3, 1, 1, 1, 2, 3, 3, 2, 2, 2, 3, 1, 2, 2, 1, 3, 1]),
#   'DD2': array([4, 2, 1, 2, 4, 4, 2, 4, 3, 4, 4, 1, 3, 3, 4, 4, 4, 4, 3, 3, 2, 4])}

# score: 116

In [2]:
import numpy as np
import random

In [3]:
import pandas as pd

In [4]:

popu_size = 50 #number of selections (each citizen is a selection)
cross_change = 5 #change {n} bad points 
mutate_num = 3
days_num = 31   #solve selection of 31 days

num_iter = 100
expect_score = 90

eliminate_rate = 0.3           
 ## how the population will grow after selection. For now it will be growing to the original popu num (0.3 is in respect to orgininal number)
crossover_rate = eliminate_rate   

columns = ['BS1', 'BS2', 'DD1', 'DD2']

In [5]:
10 % 3

1

In [6]:
# does np.random.shuffle give completely random shuffling? Or are there any elements repetitive?

# {'BS1': array([3, 2, 2, 4, 3, 3, 2, 3, 1, 4, 1, 4, 2, 2, 2, 3, 3, 1, 4]),
#  'BS2': array([4, 4, 4, 2, 1, 4, 4, 4, 3, 3, 4, 2, 2, 2, 3, 3, 1]),
#  'DD1': array([3, 2, 3, 3, 1, 2, 2, 3, 2, 2, 1, 3, 1, 2, 1, 3, 1, 3, 1, 3, 1, 2]),
#  'DD2': array([4, 3, 4, 4, 4, 2, 3, 1, 4, 2, 3, 3, 4, 4, 2, 4, 3, 1, 4, 4, 4, 2])}

def fitness(selection):
    A_s = selection['BS1']
    B_s = selection['BS2']
    C_s = selection['DD1']
    D_s = selection['DD2']
    score = 0
    lA = len(A_s)
    lB = len(B_s)
    lC = len(C_s)
    lD = len(D_s)
    lmin = min(lA, lB, lC, lD)
    for i in range(days_num):
        if i >= lmin:
            score += row_score(A_s[i % lA], B_s[i % lB], C_s[i % lC], D_s[i % lD])
            ## score = score + row_score
        else: 
            score += row_score(A_s[i], B_s[i], C_s[i], D_s[i])
    return score

def row_score(a, b, c, d):
    thisset = {a, b, c, d}
    return len(thisset)

# find a row with score less than 4:
## best_row_score default to be 4 in this problem
## temp_score as return value --- only for testing now. DELETE soon
def row_low_score(selection, best_row_score=4):
    A_s = selection['BS1']
    B_s = selection['BS2']
    C_s = selection['DD1']
    D_s = selection['DD2']
    lA = len(A_s)
    lB = len(B_s)
    lC = len(C_s)
    lD = len(D_s)
    
    iteration = 0 ##to ensure that this does not run forever
    
    #### 116 -> có 8 hàng 3 điểm -> xác suất để chọn 1 hàng 3 điểm là 8/30 -> dự kiến 4 lần sẽ tìm được
    while (iteration <= days_num * 2):
        index = random.randint(0, days_num)
        temp_score = row_score(A_s[index % lA], B_s[index % lB], C_s[index % lC], D_s[index % lD])
        if temp_score < best_row_score:
            return index   #temp_score only for testing now
        iteration = iteration + 1
    
    #in case not found any row with low score
    return 0

def rand_column_name():
    rand_key = random.randint(0, len(columns) - 1)
    return columns[rand_key]

def rand_column_index(column):
    rand_id = random.randint(0, len(column) - 1)
    return rand_id

# a column, i.e. BS1 or BS2 or DD1 or DD2
# gen_column(10, 20, 30, 40)
# gen_column()
def gen_column(a1=1, a2=1, a3=1, a4=1):
    # A = [1] * 3 + [2] * 6 + [3] * 6 + [4] * 4
    col = np.array([1] * a1 + [2] * a2 + [3] * a3 + [4] * a4)
    np.random.shuffle(col)
    return col

def gen_selection():
    select = {}
    #numbers are hard-coded here (because of this specific problem). Change here if needed.
    select['BS1'] = gen_column(3, 6, 6, 4)
    select['BS2'] = gen_column(2, 4, 4, 7)
    select['DD1'] = gen_column(7, 7, 8, 0)
    select['DD2'] = gen_column(2, 4, 5, 11)
    return select

# for each selection, keep info of just A, B, C, D (raw - NOT A_s, B_s, C_s, D_s -> )
def gen_population(popu_size=10):
    popu = []
    
    for i in range(popu_size):
        # array concatenation
        popu = popu + [gen_selection()]
    
    return popu


# map(): thực hiện 1 hàm nào đó trên từng phần tử của array
# map(lambda ptu: f(ptu), arr)   -> trả về 1 array của kết quả
    
def population_fitness(popu):
    return list(map(lambda sel: fitness(sel), popu))

def mean_fitness(popu):
    return sum(population_fitness(popu)) / len(popu)

def best_fitness(popu):
    return max(population_fitness(popu))

# eliminate 30% ({eliminate_rate}) worst-performing selections
def selection(popu, eliminate_rate=0.1):
    indices_eli = list(worst_indices(population_fitness(popu), eliminate_rate))  #các index để loại bỏ
    indices_eli.sort(reverse=True)
    #[50, 20, 40, 30] -> cần xóa 2
    #danh sách cần xóa: [1, 3] -> [3, 1]
    # Xoá 1: còn [50, 40, 30]
    
    for i in range(len(indices_eli)):
        popu.pop(indices_eli[i])

# worst selections for eliminating(by index)
# lst_fitness (population fitness), for example: [86, 83, 88, 85, 83]
def worst_indices(lst_fitness, eliminate_rate=0.1):
    lst_sorted = np.argsort(lst_fitness)
    eliminate_num = int(len(lst_fitness) * eliminate_rate)
    return lst_sorted[:eliminate_num]

#######################
######DONE crossover!

# Ý tưởng: tìm MỖI selection tốt -> làm cho nó tốt lên 
# Mục tiêu: Hồi sinh popu về dân số ban đầu
# Ý tưởng gốc: tìm giao giữa 2 selection tốt -> tạo thành selection tốt hơn
def crossover(popu, origin_population=50, bound_iter=4000):
    iteration = 0   # to ensure that don't run forever in case cannot find anything else
    while (len(popu) < origin_population and iteration <= bound_iter):   #should have used crossover_rate, but use popu_size for now (for simplicity)
        # chọn random sel để crossover (đáng ra chọn sel tốt nhất)
        rand_sel_id = random.randint(0, len(popu) - 1)
        new_sel = indi_breeding(popu[rand_sel_id])
        if not (new_sel is None):
            popu.append(new_sel)
            iteration = 0
        else:
            iteration = iteration + 1
            
#### tăng fitness cho 1 selection
# breeding_strength: đảo bao nhiêu vị trí trong selection
def indi_breeding(sel, breeding_strength=1):
    bad_row = row_low_score(sel)   # tìm 1 hàng nào đó điểm chưa tối đa 
        #ignoring case `bad_row = 0` for ease of coding
    col_rand = rand_column_name()     
    #[i, j]: vị trí i cần đổi trong cột j
    # --> tìm [u, j] để đảo cho [i, j]
    bad_row = bad_row % len(sel[col_rand]) # bad_row can be greater than 22, so need modulo-divide first
    iteration = 0  #iteration makes thing haunt when there is no expected answer (probabilistically)
    while (iteration <= len(sel[col_rand])):   
        id_rand = rand_column_index(sel[col_rand])
        if sel[col_rand][id_rand] != sel[col_rand][bad_row]: # nếu giá trị [i, j] != [u, j]
            new_col = swap_cell(sel[col_rand], bad_row, id_rand)
            new_sel = sel.copy()
            new_sel[col_rand] = new_col  #### tạo thành selection mới: new_sel, trong đó [i, j] đảo với [u, j]
            #implement condition: STRICTLY GREATER than old selection
            if fitness(new_sel) > fitness(sel):
                return new_sel    
        iteration = iteration + 1
    #in case can not change anything to increase score
    return None


# a cell is a position in a column
def swap_cell(column_origin, index_from=0, index_to=0):
    column = column_origin.copy()
    temp = column[index_to]
    column[index_to] = column[index_from]
    column[index_from] = temp
    return column

def swap_cell_origin(column_origin, index_from=0, index_to=0):
    temp = column_origin[index_to]
    column_origin[index_to] = column_origin[index_from]
    column_origin[index_from] = temp
    return column_origin

#################################
####### CONTINUE: mutating 

# mutate_rate means how many selections get mutated
# mutate_strength: bao nhiêu phần tử trong selection sẽ bị đảo
##### Biến dị: đảo một số phần tử trong selection một cách ngẫu nhiên
def mutate(popu, mutate_rate=0.1, mutate_strength=0.1):
    num_mutate = int(len(popu) * mutate_rate)
    count = 0
    
    while (count < num_mutate):
        rand_sel_id = random.randint(0, len(popu) - 1)
        indi_mutate(popu[rand_sel_id], mutate_strength)    #second param for indi_mutate is hardcoded - change here if needed
        count += 1
        
# mutate_strength means how many pairs get swaped in a selection
def indi_mutate(sel, mutate_strength=0.1):
    # num_swap: tổng số phần tử trong tất cả các cột * mutate_strength (vd: 80 * 0.1 = 8)
    num_swap = int( sum(list(map(lambda k: len(sel[k]), sel))) * mutate_strength )
    count = 0
    
    while (count < num_swap):
        # chọn ra vị trí [i, j]
        col_rand = rand_column_name()     
        id1_rand = rand_column_index(sel[col_rand])
        id2_rand = id1_rand
        # chọn u
        while (sel[col_rand][id1_rand] == sel[col_rand][id2_rand]):
            # chọn ra vị trí [u, j] với u != i
            id2_rand = rand_column_index(sel[col_rand])
        
        swap_cell_origin(sel[col_rand], id1_rand, id2_rand)
        count += 1        

########################################
## selection -> crossover -> mutation
# done selection and crossover. Testing with them
# run genetic algorithm
def run_GA(popu_size=50, eliminate_rate=0.3, bound_iter=4000, mutate_rate=0.3, mutate_strength=0.1, aim=100):
    popu = gen_population(popu_size)
    prev_score = 0
    score = 1
    count = 0 #for testing
    while (best_fitness(popu) < aim):
        selection(popu, eliminate_rate)
#         prev_score = mean_fitness(popu)
# crossover: chưa tối ưu
        crossover(popu, popu_size, bound_iter)
#         score = mean_fitness(popu)
# mutate: chưa tối ưu -> kết quả chưa tốt
#         mutate(popu, mutate_rate, mutate_strength)   # 2nd param is hardcoded - temporarily for case popu_size=10
        count += 1
    return (popu,count)

In [9]:
popu, _ = run_GA(50, 0.3, 4000, 0, 0, 115)

In [11]:
population_fitness(popu)

[112,
 112,
 112,
 112,
 112,
 113,
 112,
 112,
 112,
 112,
 112,
 112,
 112,
 112,
 112,
 113,
 113,
 112,
 113,
 113,
 113,
 113,
 113,
 113,
 113,
 114,
 113,
 113,
 113,
 113,
 113,
 113,
 113,
 114,
 113,
 114,
 114,
 114,
 114,
 113,
 114,
 113,
 114,
 113,
 114,
 113,
 115,
 114,
 114,
 113]

In [13]:
popu_ex = gen_population(5)
popu_ex

[{'BS1': array([3, 2, 2, 2, 3, 2, 3, 4, 4, 4, 1, 4, 3, 2, 1, 2, 1, 3, 3]),
  'BS2': array([3, 4, 2, 2, 4, 4, 1, 4, 2, 3, 4, 4, 3, 1, 4, 3, 2]),
  'DD1': array([1, 2, 2, 3, 1, 3, 1, 3, 1, 2, 2, 2, 2, 1, 3, 1, 3, 3, 3, 2, 3, 1]),
  'DD2': array([3, 2, 4, 4, 1, 4, 3, 4, 3, 4, 1, 4, 4, 4, 3, 4, 4, 2, 2, 4, 3, 2])},
 {'BS1': array([3, 4, 3, 3, 1, 3, 2, 1, 2, 2, 2, 2, 4, 1, 3, 3, 4, 4, 2]),
  'BS2': array([2, 3, 2, 3, 4, 4, 4, 4, 1, 2, 3, 4, 4, 3, 4, 2, 1]),
  'DD1': array([1, 2, 3, 3, 2, 3, 3, 1, 2, 2, 3, 1, 1, 2, 2, 3, 1, 2, 1, 3, 1, 3]),
  'DD2': array([4, 2, 3, 4, 4, 2, 1, 4, 3, 4, 3, 2, 4, 2, 1, 3, 4, 4, 4, 4, 3, 4])},
 {'BS1': array([3, 1, 4, 3, 4, 2, 3, 2, 2, 1, 3, 2, 3, 4, 2, 2, 1, 4, 3]),
  'BS2': array([1, 4, 4, 4, 2, 3, 2, 3, 2, 4, 3, 2, 4, 4, 3, 4, 1]),
  'DD1': array([2, 3, 3, 1, 3, 2, 2, 2, 2, 3, 2, 2, 3, 1, 3, 3, 1, 3, 1, 1, 1, 1]),
  'DD2': array([3, 2, 4, 4, 4, 4, 4, 3, 4, 1, 3, 1, 4, 2, 4, 3, 4, 3, 4, 2, 2, 4])},
 {'BS1': array([2, 3, 2, 1, 1, 2, 2, 4, 2, 3, 4, 2, 1, 4, 3, 